Task
- Add in other uestions
-- Refactor code for the aggregation of timely activity
- Fix the plots
- Start on EDA
- Model selection

Background Information on Dataset: BLAH BLAH BLAH

Importing Packages

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib
import plotly
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime

EDA

Load in dataset

Aside: Let's measure how long it takes to load in our dataset: using pandas and dask

In [ ]:
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()
df = pd.read_parquet("Data/Large/articles.parquet")
profiler.disable()

profiler_stats = pstats.Stats(profiler)
profiler_stats.print_stats()

In [ ]:
profiler = cProfile.Profile()
profiler.enable()
df = dd.read_parquet("Data/Large/articles.parquet", engine = "fastparquet")
profiler.disable()

profiler_stats = pstats.Stats(profiler)
profiler_stats.print_stats()

Even though the dataframe took 1.464 seconds to load using pandas, dask took 0.054 seconds. This is a huge speedup ~ 27x

In [2]:
#Load in various dataframes
## Articles
df_art = pd.read_parquet("Data/Small/articles.parquet")

## Behaviors
df_bev = pd.read_parquet("Data/Small/train/behaviors.parquet")

## History
df_his = pd.read_parquet("Data/Small/train/history.parquet")



Join the data sources

In [3]:
# Convert datatype of column first
df_bev['article_ids_clicked'] = df_bev['article_ids_clicked'].apply(lambda x: x[0])

In [4]:
# Join bevhaiors to article
df= df_bev.join(df_art.set_index("article_id"), on = "article_ids_clicked")

# Join bevhaiors to history 
df= df.join(df_his.set_index("user_id"), on = "user_id")

# Drop all other dataframes from me
df_bev = []
df_his = []
df_art = []

Visualizations

In [6]:
def plotly_index_values(groupby_object):
    """Takes in a clause from a pandas groupby statement and returns X and Y variables used for plotting"""

    index = groupby_object.index
    values = groupby_object.values

    return index, values



Lets calculate the unique users for hourly, daily, and day of the week. Let's use a subset of the data until we know our plots are very good

In [70]:
# Let's make subsets for each user_id and then populate the frequency based on that

## Get all unique ids in a list

unique_user_ids = df['user_id'].values[0:1000]
unique_user_ids = set(unique_user_ids)


# Create dictionaries
unique_users_daily_freq = {}
unique_users_hourly_freq = {}
unique_users_dayofweek_freq = {}
unique_users_weekly_freq = {}

for id in unique_user_ids:
    # Get the subset of that user id
    tmp_df = df[df['user_id'] == id]

    # Now lets go through each and populate the unique dates, hours and day of the week for each user
    dates = []
    hours = []
    dayofweek = []
    week = []
    indices = np.array(tmp_df.index)
    for i in indices:
        tmp_datetime = pd.DatetimeIndex(tmp_df['impression_time_fixed'][i])
        tmp_date = tmp_datetime.date
        tmp_time = tmp_datetime.time
        tmp_dayofweek = tmp_datetime.day_of_week
        tmp_week = tmp_datetime.isocalendar().week

        for j in tmp_date:
            dates.append(j)
        
        for k in tmp_time:
            hours.append(k)
        
        for l in tmp_dayofweek:
            dayofweek.append(l)
        
        for m in tmp_week:
            week.append(m)

    # Get rid of duplicate values
    unique_dates = list(set(dates))
    unique_hours = list(set(hours))
    unique_dayofweek = list(set(dayofweek))
    unique_week = list(set(week))

    # Convert to string
    unique_dates = [x.strftime('%m/%d/%Y') for x in unique_dates]
    unique_hours = [x.hour for x in unique_hours]

   # unique_dayofweek = [x for x in unique_dayofweek]


    # Populate our unique_user_daily_freq dict
    for i in unique_dates:
        
        if i not in unique_users_daily_freq:
            unique_users_daily_freq[i] = 1
        else:
            unique_users_daily_freq[i] +=1
        

    # Populate hourly activity
    for j in unique_hours:

        if j not in unique_users_hourly_freq:
            unique_users_hourly_freq[j] = 1
        else:
            unique_users_hourly_freq[j] +=1

    # Populate dayofweeka activity
    for k in unique_dayofweek:

        if k not in unique_users_dayofweek_freq:
            unique_users_dayofweek_freq[k] = 1
        else:
            unique_users_dayofweek_freq[k] +=1
    

    # Populate dayofweeka activity
    for l in unique_week:

        if l not in unique_users_weekly_freq:
            unique_users_weekly_freq[l] = 1
        else:
            unique_users_weekly_freq[l] +=1
    
            
    

In [71]:
# What does the daily user activity look like?
indices = [x for x in unique_users_daily_freq.keys()]
values = [x for x in unique_users_daily_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'lines+markers'
                         )

)

fig.show()


In [75]:
# What does the hourly user activity look like?
indices = [x for x in unique_users_hourly_freq.keys()]
values = [x for x in unique_users_hourly_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'markers'
                         )

)

fig.show()

In [74]:
# What does the weekly user activity look like?
indices = [x for x in unique_users_weekly_freq.keys()]
values = [x for x in unique_users_weekly_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'lines+markers'
                         )

)

fig.show()

In [76]:
# What does the weekly user activity look like?
indices = [x for x in unique_users_dayofweek_freq.keys()]
values = [x for x in unique_users_dayofweek_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x = indices, y = values,
                         mode = 'markers'
                         )

)

fig.show()

What is the distribution of topics people view?

In [23]:
# Let's get unique_user_ids topics

## Get all unique ids in a list

unique_user_ids = df['user_id'].values[0:1000]

# Create dictionaries
unique_users_topics_freq= {}

for id in unique_user_ids:
    # Get the subset of that user id
    tmp_df = df[df['user_id'] == id]

    # Now lets go through each topic
    topics = []
    indices = np.array(tmp_df.index)
    for i in indices:
        tmp_topics = tmp_df['topics'][i]

        for j in tmp_topics:
            topics.append(j)
        

    # Get rid of duplicate values
    unique_topics = list(set(topics))

    
    # Populate our dict
    for i in unique_topics:
        
        if i not in unique_users_topics_freq:
            unique_users_topics_freq[i] = 1
        else:
            unique_users_topics_freq[i] +=1


# What does the distribution of topics look like lets sort it?
sorted_topic_freq = dict(sorted(unique_users_topics_freq.items(), key = lambda x: x[1], reverse = True))


indices = [x for x in sorted_topic_freq.keys()]
values = [x for x in sorted_topic_freq.values()]

fig = go.Figure()

# Add traces
fig.add_trace(go.Bar(x = indices, y = values,
                         
                         )

)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.show()




In [ ]:
## How long is the average session length per user

In [43]:
# For each session, what was the average readtime 

# Distribution blot histogram (bin it)
df.groupby(by= 'session_id')['read_time'].mean()
    

In [83]:
## For each article, what was each users scroll percentage + read time? 


232887

In [ ]:
# Average scroll time across each user

In [ ]:
# Find the average scroll percentages across each topic  (Can be related to whether a topic doesnt require too much reading has visualizations)

## Group by user ID
### Get a frequency count of each topic and average scroll percentages 

In [ ]:
# Biggest thing is user engagement : Bigger User Engagement -> More eveneue
## We need to maximize the amount of ads these guys are viewing -> this leads on to them clicking on new articles for ads
### So, let's not make article length too short so that people can maximize their session lengths with a lot of articles!